# Imports

In [60]:
#Always make all imports in the first cell of the notebook, run them all once.
import numpy as np
from Preproccessing import preproccessing
from FeatureExtraction import extract_features
from IO import ReadDataSet
from Phase1 import Phase1
from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import f1_score, accuracy_score
import cv2
import glob
import pickle
import os
import matplotlib.pyplot as plt
import math
from PIL import Image, ImageDraw, ImageFont

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read Dataset

In [47]:
'''
1.a) READ THE DATASET
'''
x,y = ReadDataSet()

'''
 1.b) Splitting the data 
        Training Set : 70%
        Test Set : 15 %
'''
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=42)

# PHASE-1
Training Phase

In [48]:
model = Phase1(x,y)

In [49]:
# # save the model to disk
# filename = 'finalized_model.sav'
# pickle.dump(model, open(filename, 'wb'))

# # some time later...

# # load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

# PHASE-2
Testing Phase

In [50]:
# read test data from file
x_test = []
# for filename in glob.glob('./test/*.*'):
#     print(filename)
#     img = cv2.imread(filename) 
#     x_test.append(img)
path = "./test"
i=0
list_dir=[int(file.split(".")[0]) for file in os.listdir(path)]
list_dir.sort()
for fname in list_dir:    
    img = cv2.imread(path + '/' + str(fname)+".png")
    x_test.append(img)
    i += 1

y_test = None
with open('./ground_truth.txt', 'rb') as gt_file:
    y_test = [ int(line)  for line in gt_file.readlines()]
    y_test = np.array(y_test)



In [57]:
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        plt.axis('off')
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()


TypeError: cannot unpack non-iterable NoneType object

In [53]:
y_pred = np.zeros(len(x_test))
runningTime = np.zeros(len(x_test))

# 1) Loop on test data
total_start = time.time()
for i in range(0,len(x_test)):
    try:    
        #3.a) start timer
        start = time.time()
        #3.b) Preprocess the test image
        x_test[i] = preproccessing(x_test[i].astype('uint8'))
        # 3.c) Extract features from the test image
        x_test[i] = extract_features(x_test[i])
        # 3.d) Classify the test image
        y_pred[i] = model.predict([x_test[i]])

        # 3.e) Stop timer
        runningTime[i] = round(time.time()-start,2)
    except:
        runningTime[i] = round(time.time()-start,2)
        y_pred[i] = -1
        continue

total_running = time.time() - total_start


In [54]:
print(len(y_test))
print(len(x_test))
print(len(y_pred))

178
178
178


In [55]:
# calculate model accuracy and F1Score
y_pred = np.array(y_pred)
y_test = np.array(y_test)
print('runningTime in sec =', total_running)
print('Accuracy = ' , (np.sum(y_pred == y_test)/len(y_test))*100)
print('F1Score = ',  f1_score(y_test, y_pred, average='weighted') *100)

runningTime in sec = 11.610106706619263
Accuracy =  68.53932584269663
F1Score =  64.93022464184058


In [56]:
# print(x_test)
